# 02. LangChain 캐싱(Caching) 가이드

이 노트북에서는 LangChain의 캐싱 기능을 사용하여 LLM 호출 비용을 절감하고 성능을 향상시키는 방법을 배웁니다.

## 목차
1. LangSmith 설정 및 연결
2. InMemory Cache
3. SQLite Cache
4. 성능 비교 및 모범 사례

## 개요

캐싱은 동일한 입력에 대한 LLM의 응답을 저장하여 재사용하는 기법입니다. 이를 통해:
- **비용 절감**: 동일한 쿼리에 대해 API 호출을 반복하지 않음
- **응답 속도 향상**: 캐시된 결과는 즉시 반환됨
- **일관성 보장**: 동일한 입력에 대해 항상 같은 응답을 받음

## 1. 환경 설정 및 LangSmith 연결

먼저 필요한 패키지를 임포트하고 환경 변수를 설정합니다.

## 0. 필수 패키지 설치

이 노트북을 실행하기 전에 필요한 패키지들을 설치해야 합니다.

### 필요한 패키지
- `langchain_community`: 캐싱 기능 제공
- `langsmith`: LangSmith 추적 및 모니터링
- `langchain_google_genai`: Google Gemini 모델 사용
- `python-dotenv`: 환경 변수 관리

### 설치 명령어
```bash
# uv를 사용하는 경우
uv add langchain_community langsmith langchain_google_genai python-dotenv

# pip를 사용하는 경우
pip install langchain_community langsmith langchain_google_genai python-dotenv
```

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.globals import set_llm_cache
from langchain_community.cache import InMemoryCache, SQLiteCache
import time
from langchain.schema import HumanMessage, SystemMessage

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수 확인
print("🔍 환경 변수 확인 중...")

# Google API 키 확인
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
print("✅ Google API Key: 설정됨")

# LangSmith API 키 확인 (선택사항)
langsmith_api_key = os.getenv("LANGCHAIN_API_KEY")
if langsmith_api_key:
    print("✅ LangSmith API Key: 설정됨")
    # LangSmith 설정
    os.environ["LANGCHAIN_TRACING_V2"] = "true"  # LangSmith 추적 활성화
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "cache-demo"  # 프로젝트 이름 설정
    
    print(f"📊 LangSmith 프로젝트: {os.environ['LANGCHAIN_PROJECT']}")
    print(f"🔄 LangSmith 추적 활성화: {os.environ['LANGCHAIN_TRACING_V2']}")
    print("🌐 LangSmith 대시보드: https://smith.langchain.com/")
else:
    print("⚠️  LangSmith API Key가 설정되지 않았습니다.")
    print("   LangSmith 추적 없이 계속 진행합니다.")
    print("   추적을 원한다면 .env 파일에 LANGCHAIN_API_KEY를 추가하세요.")

print("\n✅ 환경 설정 완료!")

In [ ]:
# 🔍 패키지 설치 상태 확인
try:
    import langchain_community
    print("✅ langchain_community 설치됨")
    print(f"   버전: {langchain_community.__version__}")
except ImportError as e:
    print("❌ langchain_community 패키지 설치 필요")
    print(f"   에러: {e}")
    print("   해결: uv add langchain_community")

try:
    import langsmith
    print("✅ langsmith 설치됨")
    print(f"   버전: {langsmith.__version__}")
except ImportError as e:
    print("❌ langsmith 패키지 설치 필요")
    print(f"   에러: {e}")
    print("   해결: uv add langsmith")

try:
    from langchain_community.cache import InMemoryCache, SQLiteCache
    print("✅ 캐시 클래스 임포트 성공")
except ImportError as e:
    print("❌ 캐시 클래스 임포트 실패")
    print(f"   에러: {e}")

try:
    from langchain.globals import set_llm_cache
    print("✅ LLM 캐시 설정 함수 임포트 성공")
except ImportError as e:
    print("❌ LLM 캐시 설정 함수 임포트 실패")
    print(f"   에러: {e}")

### 🔗 LangSmith 프로젝트 연결 가이드

LangSmith는 LangChain 애플리케이션의 디버깅, 테스팅, 모니터링을 위한 플랫폼입니다.

#### 📋 단계별 설정 방법

**1단계: LangSmith 계정 생성**
- [LangSmith 웹사이트](https://smith.langchain.com/)에 접속
- 계정 생성 또는 로그인
- 새 프로젝트 생성

**2단계: API 키 발급**
- LangSmith 대시보드에서 Settings → API Keys 메뉴
- "Create API Key" 클릭
- API 키 복사 (한 번만 표시됨!)

**3단계: .env 파일 설정**
```env
# .env 파일에 추가
GOOGLE_API_KEY=your_google_api_key
LANGCHAIN_API_KEY=your_langsmith_api_key
```

**4단계: 환경 변수 설정 (아래 코드셀에서 자동 처리)**
- `LANGCHAIN_TRACING_V2="true"`: 추적 활성화
- `LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"`: API 엔드포인트
- `LANGCHAIN_PROJECT="your-project-name"`: 프로젝트 이름

#### 🎯 LangSmith의 주요 기능

1. **실시간 추적**: 모든 LLM 호출 모니터링
2. **성능 분석**: 응답 시간, 토큰 사용량, 비용 추적
3. **디버깅**: 프롬프트와 응답 체인 시각화
4. **A/B 테스팅**: 다른 모델/프롬프트 성능 비교
5. **데이터셋 관리**: 테스트 케이스 저장 및 평가

#### ⚠️ 주의사항
- API 키는 절대 코드에 직접 입력하지 마세요
- .env 파일을 .gitignore에 추가하세요
- 무료 플랜은 월 5,000개 트레이스 제한

In [ ]:
# Gemini 모델 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,  # 일관된 응답을 위해 0으로 설정
    max_output_tokens=100
)

# 테스트 메시지
test_message = "파이썬의 주요 특징 3가지를 간단히 설명해주세요."

print("LLM 모델 초기화 완료!")

## 2. InMemory Cache

InMemory Cache는 응답을 메모리에 저장하는 가장 간단한 캐싱 방법입니다.

### 특징
- **빠른 속도**: 메모리에서 직접 읽기/쓰기
- **휘발성**: 프로그램 종료 시 캐시 데이터 손실
- **용도**: 개발/테스트 환경, 단기 실행 애플리케이션

In [ ]:
############################################################################
# InMemory Cache
############################################################################
# ⭐⭐⭐ InMemory Cache 설정
set_llm_cache(InMemoryCache())

print("InMemory Cache를 사용한 LLM 호출 테스트")
print("-" * 50)

# 첫 번째 호출 (캐시 미스)
start_time = time.time()
response1 = llm.invoke(test_message)
first_call_time = time.time() - start_time

print(f"첫 번째 호출 (캐시 미스):")
print(f"응답 시간: {first_call_time:.2f}초")
print(f"응답: {response1.content[:100]}...")
print()

# 두 번째 호출 (캐시 히트)
start_time = time.time()
response2 = llm.invoke(test_message)
second_call_time = time.time() - start_time

print(f"두 번째 호출 (캐시 히트):")
print(f"응답 시간: {second_call_time:.2f}초")
print(f"응답: {response2.content[:100]}...")
print()

print(f"속도 향상: {first_call_time/second_call_time:.0f}배 빠름")

### InMemory Cache 고급 사용법

In [ ]:
############################################################################
# 다양한 쿼리로 캐시 테스트
############################################################################
queries = [
    "파이썬의 주요 특징 3가지를 간단히 설명해주세요.",  # 이미 캐시됨
    "자바스크립트의 주요 특징을 설명해주세요.",
    "파이썬의 주요 특징 3가지를 간단히 설명해주시기를 바랍니다.",  # 캐시에서 못 가져옴
    "머신러닝이란 무엇인가요?",
    "자바스크립트의 주요 특징을 설명해주시죠."  # 캐시에서 못 가져옴
]

# 질문이 조금이라도 다르면 캐시를 활용하지 못함.
# 예: "설명해주세요" vs "설명해주시기를 바랍니다"
# 캐시를 제대로 활용하기 위해서는 질문 문장을 일반화하는 로직 필요

print("다중 쿼리 캐시 테스트")
print("=" * 60)

for i, query in enumerate(queries, 1):
    start_time = time.time()
    response = llm.invoke(query)
    elapsed_time = time.time() - start_time
    
    # 캐시 상태 판단
    cache_status = "캐시 히트" if elapsed_time < 0.1 else "캐시 미스"
    
    print(f"\n쿼리 {i}: {query[:30]}...")
    print(f"상태: {cache_status}")
    print(f"응답 시간: {elapsed_time:.3f}초")
    print(f"응답 미리보기: {response.content[:50]}...")

## 3. SQLite Cache

SQLite Cache는 응답을 SQLite 데이터베이스에 저장하여 영구 보존합니다.

### 특징
- **영구 저장**: 프로그램 종료 후에도 캐시 유지
- **파일 기반**: `.db` 파일로 관리
- **용도**: 프로덕션 환경, 장기 실행 애플리케이션, 캐시 공유 필요 시

In [ ]:
############################################################################
# SQLite Cache 설정
############################################################################
sqlite_cache_path = "langchain_cache.db"
set_llm_cache(SQLiteCache(database_path=sqlite_cache_path))

print("SQLite Cache를 사용한 LLM 호출 테스트")
print("-" * 50)

# 새로운 테스트 쿼리
test_queries = [
    "딥러닝과 머신러닝의 차이점은?",
    "REST API란 무엇인가요?",
    "딥러닝과 머신러닝의 차이점은?"  # 동일 쿼리 반복
]

for idx, query in enumerate(test_queries, 1):
    print(f"\n쿼리 {idx}: {query}")
    
    start_time = time.time()
    response = llm.invoke(query)
    elapsed_time = time.time() - start_time
    
    # 캐시 상태 판단
    if elapsed_time < 0.1:
        print("📦 캐시에서 로드됨!")
    else:
        print("🔄 새로운 API 호출")
    
    print(f"응답 시간: {elapsed_time:.3f}초")
    print(f"응답: {response.content[:80]}...")

# DB 파일 정보 출력
import os
if os.path.exists(sqlite_cache_path):
    file_size = os.path.getsize(sqlite_cache_path) / 1024  # KB
    print(f"\n\n💾 SQLite 캐시 파일 크기: {file_size:.2f} KB")

# SQLite 캐시 연결 정리
print(f"\n🧹 SQLite 캐시 연결 정리 중...")
from langchain.globals import get_llm_cache

current_cache = get_llm_cache()
if current_cache and hasattr(current_cache, 'engine'):
    current_cache.engine.dispose()
    print("✅ SQLite 연결 정리 완료")

# 캐시 무효화 (다음 셀에서 새로운 캐시 설정을 위해)
set_llm_cache(None)
print("✅ 캐시 무효화 완료")

### SQLite 캐시 내용 확인

In [ ]:
############################################################################
# SQLite 캐시 데이터베이스 내용 조회
############################################################################
import sqlite3

# 데이터베이스 연결
conn = sqlite3.connect(sqlite_cache_path)
cursor = conn.cursor()

# 테이블 구조 확인
print("캐시 데이터베이스 구조:")
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
for table in cursor.fetchall():
    print(table[0])

print("\n\n캐시된 항목들:")
print("-" * 80)

# 캐시 내용 조회 (최근 5개)
cursor.execute("""
    SELECT prompt, response, llm, LENGTH(response) as response_length
    FROM full_llm_cache 
    ORDER BY rowid DESC 
    LIMIT 5
""")

for idx, (prompt, response, llm_type, length) in enumerate(cursor.fetchall(), 1):
    print(f"\n항목 {idx}:")
    print(f"  프롬프트: {prompt[:50]}...")
    print(f"  응답 길이: {length} 글자")
    print(f"  LLM 유형: {llm_type[:50]}...")
    print(f"  응답 미리보기: {response[:100]}...")

# 전체 캐시 항목 수
cursor.execute("SELECT COUNT(*) FROM full_llm_cache")
total_items = cursor.fetchone()[0]
print(f"\n\n총 캐시된 항목 수: {total_items}개")

conn.close()

# SQLite 캐시 연결 정리 (데이터베이스 조회 후)
print(f"\n🧹 SQLite 캐시 연결 정리 중...")
from langchain.globals import get_llm_cache

current_cache = get_llm_cache()
if current_cache and hasattr(current_cache, 'engine'):
    current_cache.engine.dispose()
    print("✅ SQLite 연결 정리 완료")

# 다음 단계를 위해 캐시 무효화
set_llm_cache(None)
print("✅ 캐시 무효화 완료")

## 4. 캐시 성능 비교 및 모범 사례

In [ ]:
############################################################################
# 캐시 성능 비교
############################################################################
def benchmark_cache(cache_type, cache_instance, queries, runs=3):
    """캐시 성능을 측정하는 함수"""
    set_llm_cache(cache_instance)
    
    results = {
        "cache_type": cache_type,
        "first_call_times": [],
        "cached_call_times": [],
        "total_time": 0
    }
    
    try:
        for run in range(runs):
            for query in queries:
                # 첫 호출 (캐시 미스)
                start = time.time()
                llm.invoke(query)
                first_time = time.time() - start
                results["first_call_times"].append(first_time)
                
                # 캐시된 호출 (캐시 히트)
                start = time.time()
                llm.invoke(query)
                cached_time = time.time() - start
                results["cached_call_times"].append(cached_time)
        
        results["avg_first_call"] = sum(results["first_call_times"]) / len(results["first_call_times"])
        results["avg_cached_call"] = sum(results["cached_call_times"]) / len(results["cached_call_times"])
        results["speedup"] = results["avg_first_call"] / results["avg_cached_call"]
        
    finally:
        # SQLite 캐시인 경우 연결 정리
        if hasattr(cache_instance, 'engine') and cache_instance.engine:
            cache_instance.engine.dispose()
        
        # 캐시 무효화 (연결 해제)
        set_llm_cache(None)
    
    return results

# 테스트 쿼리
benchmark_queries = [
    "AI의 미래에 대해 설명해주세요.",
    "블록체인 기술의 장점은?",
    "클라우드 컴퓨팅이란?"
]

# 벤치마크 실행
print("캐시 성능 벤치마크")
print("=" * 60)

# InMemory Cache 벤치마크
inmemory_results = benchmark_cache("InMemory", InMemoryCache(), benchmark_queries)

# SQLite Cache 벤치마크 (새로운 DB 파일 사용)
sqlite_results = benchmark_cache("SQLite", SQLiteCache("benchmark_cache.db"), benchmark_queries)

# 결과 출력
for results in [inmemory_results, sqlite_results]:
    print(f"\n{results['cache_type']} Cache 성능:")
    print(f"  평균 첫 호출 시간: {results['avg_first_call']:.3f}초")
    print(f"  평균 캐시 호출 시간: {results['avg_cached_call']:.3f}초")
    print(f"  속도 향상: {results['speedup']:.0f}배")

# 정리 (안전한 파일 삭제)
import time
import gc

def safe_remove_file(file_path, max_attempts=5, delay=2):
    """파일을 안전하게 삭제하는 함수"""
    # 가비지 컬렉션 강제 실행으로 참조 정리
    gc.collect()
    time.sleep(1)
    
    for attempt in range(max_attempts):
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"✅ {file_path} 삭제 완료")
                return True
        except PermissionError:
            if attempt < max_attempts - 1:
                print(f"⚠️ {file_path} 삭제 시도 {attempt + 1}/{max_attempts} - 잠시 후 재시도...")
                gc.collect()  # 매 시도마다 가비지 컬렉션
                time.sleep(delay)
            else:
                print(f"❌ {file_path} 삭제 실패: 파일이 사용 중입니다.")
                print(f"   수동으로 삭제하거나 프로그램 종료 후 삭제하세요.")
                return False
        except Exception as e:
            print(f"❌ {file_path} 삭제 중 오류: {e}")
            return False
    return False

# 벤치마크용 캐시 파일 정리
safe_remove_file("benchmark_cache.db")

### 캐시 사용 모범 사례

1. **개발 환경**: InMemory Cache 사용
   - 빠른 프로토타이핑
   - 테스트 중 캐시 초기화 용이

2. **프로덕션 환경**: SQLite Cache 또는 Redis Cache 사용
   - 영구 저장 필요
   - 여러 프로세스 간 캐시 공유

3. **캐시 무효화 전략**
   - 시간 기반: 일정 시간 후 캐시 만료
   - 이벤트 기반: 데이터 변경 시 캐시 삭제
   - 수동: 필요 시 캐시 파일 삭제

4. **주의사항**
   - 민감한 정보는 캐시하지 않기
   - 캐시 크기 모니터링
   - 정기적인 캐시 정리

### 실전 예제: 대화형 챗봇에서 캐시 활용

In [ ]:
# 실전 예제: FAQ 챗봇 with 캐싱
from langchain.prompts import ChatPromptTemplate

# SQLite 캐시 설정 (영구 저장)
set_llm_cache(SQLiteCache(database_path="faq_cache.db"))

# FAQ 템플릿
faq_template = ChatPromptTemplate.from_template(
    "당신은 친절한 고객 지원 챗봇입니다. 다음 질문에 간결하고 정확하게 답변해주세요: {question}"
)

# 자주 묻는 질문들
faqs = [
    "환불 정책이 어떻게 되나요?",
    "배송 기간은 얼마나 걸리나요?",
    "회원가입 혜택은 무엇인가요?",
    "환불 정책이 어떻게 되나요?",  # 중복 질문
    "배송 기간은 얼마나 걸리나요?"   # 중복 질문
]

print("FAQ 챗봇 시뮬레이션 (캐싱 활용)")
print("=" * 60)

total_time = 0
cached_count = 0

for i, question in enumerate(faqs, 1):
    print(f"\n질문 {i}: {question}")
    
    # 프롬프트 생성
    prompt = faq_template.format_messages(question=question)
    
    # 응답 시간 측정
    start_time = time.time()
    response = llm.invoke(prompt)
    elapsed_time = time.time() - start_time
    total_time += elapsed_time
    
    # 캐시 상태 표시
    if elapsed_time < 0.1:
        print("✅ 캐시된 응답 사용")
        cached_count += 1
    else:
        print("🔄 새로운 응답 생성")
    
    print(f"응답 시간: {elapsed_time:.3f}초")
    print(f"답변: {response.content[:100]}...")

# 통계
print(f"\n\n📊 캐싱 통계:")
print(f"- 총 질문 수: {len(faqs)}")
print(f"- 캐시 히트: {cached_count}회")
print(f"- 캐시 히트율: {cached_count/len(faqs)*100:.0f}%")
print(f"- 총 응답 시간: {total_time:.2f}초")
print(f"- 평균 응답 시간: {total_time/len(faqs):.3f}초")

# SQLite 캐시 연결 정리
print(f"\n🧹 캐시 연결 정리 중...")
from langchain.globals import get_llm_cache

# 현재 캐시 인스턴스 가져오기
current_cache = get_llm_cache()
if current_cache and hasattr(current_cache, 'engine'):
    current_cache.engine.dispose()
    print("✅ SQLite 연결 정리 완료")

# 캐시 무효화
set_llm_cache(None)
print("✅ 캐시 무효화 완료")

## 정리

이 노트북에서는 LangChain의 캐싱 기능을 배웠습니다:

1. **LangSmith 연결**: LLM 호출 추적 및 모니터링
2. **InMemory Cache**: 빠르고 간단한 메모리 기반 캐싱
3. **SQLite Cache**: 영구 저장이 가능한 데이터베이스 기반 캐싱

캐싱을 통해:
- API 호출 비용을 크게 절감
- 응답 속도를 수십~수백 배 향상
- 일관된 응답 보장

프로젝트의 요구사항에 맞는 캐싱 전략을 선택하여 효율적인 LLM 애플리케이션을 구축하세요!

### 캐시 파일 정리

In [ ]:
# 생성된 캐시 파일 정리 (선택사항)
import os

cache_files = ["langchain_cache.db", "faq_cache.db"]

for cache_file in cache_files:
    if os.path.exists(cache_file):
        print(f"캐시 파일 발견: {cache_file} ({os.path.getsize(cache_file)/1024:.2f} KB)")
        # 파일을 삭제하려면 아래 줄의 주석을 해제하세요
        safe_remove_file(cache_file)
        print(f"  -> 삭제됨")
    else:
        print(f"캐시 파일 없음: {cache_file}")